In [1]:
import sys
import time
import torch
import torch.nn.functional as F
from utils import load_data, mat2tensor, regression_loss
from model import GCN, GAT
import numpy as np
import dgl

Using backend: pytorch


In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [7]:
features_list, adjM, labels, train_val_test_idx, dl = load_data('iYO844')

In [8]:
features_list

[array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.71546936, ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.7172692 , ..., 0.        , 0.        ,
         0.        ],
        ...,
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
         0.        ],
        [0.        , 0.        , 0.7081847 , ..., 0.        , 0.        ,
         0.        ]]),
 array([[0.        , 0.        , 0.        , ..., 0.07333138, 0.01263912,
         0.01970579],
        [0.        , 0.        , 0.        , ..., 0.07504918, 0.00998594,
         0.03200828],
        [0.        , 0.        , 0.        , ..., 0.06466646, 0.00431759,
         0.01475397],
        ...,
        [0.        , 0.        , 0.        , ..., 0.08506839, 0.00357313,
         0.03697453],
        [0. 

In [9]:
features_list = [mat2tensor(features).to(device) for features in features_list]
features_list

[tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.7155,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.7173,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.7082,  ..., 0.0000, 0.0000, 0.0000]]),
 tensor([[0.0000, 0.0000, 0.0000,  ..., 0.0733, 0.0126, 0.0197],
         [0.0000, 0.0000, 0.0000,  ..., 0.0750, 0.0100, 0.0320],
         [0.0000, 0.0000, 0.0000,  ..., 0.0647, 0.0043, 0.0148],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0851, 0.0036, 0.0370],
         [0.0000, 0.0000, 0.0000,  ..., 0.0644, 0.0120, 0.0240],
         [0.0000, 0.0000, 0.0000,  ..., 0.0771, 0.0129, 0.0293]])]

In [10]:
labels

array([[2.4300e-02, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 1.7200e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.0000e+00, 0.0000e+00],
       [0.

In [13]:
labels_tensor = mat2tensor(labels).to(device)
labels_tensor

tensor([[2.4300e-02, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 1.7200e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.0000e+00, 0.0000e+00],
        [0.000

In [14]:
in_dims = [features.shape[1] for features in features_list]

In [15]:
## Set train, val, test index
labels = torch.FloatTensor(labels).to(device)
train_idx = train_val_test_idx['train_idx']
train_idx = np.sort(train_idx)
val_idx = train_val_test_idx['val_idx']
val_idx = np.sort(val_idx)
test_idx = train_val_test_idx['test_idx']
test_idx = np.sort(test_idx)

In [17]:
## Build graph
g = dgl.from_scipy(adjM+(adjM.T))
g = dgl.remove_self_loop(g)
g = dgl.add_self_loop(g)
g = g.to(device)

# Set model
num_labels = dl.labels_train['num_labels']

In [19]:
net = GCN(g, in_dims, 64, num_labels, 2, F.elu, 0.5)
net.to(device)

GCN(
  (layers): ModuleList(
    (0): GraphConv(in=64, out=64, normalization=both, activation=<function elu at 0x7f86f2ade3a0>)
    (1): GraphConv(in=64, out=64, normalization=both, activation=<function elu at 0x7f86f2ade3a0>)
    (2): GraphConv(in=64, out=2, normalization=both, activation=None)
  )
  (fc_list): ModuleList(
    (0): Linear(in_features=384, out_features=64, bias=True)
    (1): Linear(in_features=384, out_features=64, bias=True)
  )
  (dropout): Dropout(p=0.5, inplace=False)
)

In [21]:
logits = net(features_list)
print(logits[train_idx].shape)
logits[train_idx]

torch.Size([44, 2])


tensor([[-5.6322e-01, -9.8953e-01],
        [-8.6186e-01,  1.8913e-01],
        [ 2.7833e-01,  7.4442e-02],
        [ 1.1075e-03, -6.6328e-02],
        [-2.5288e-01, -6.4317e-02],
        [-3.0770e-01, -1.0815e+00],
        [-5.9276e-01, -1.3301e+00],
        [ 7.8133e-01, -4.8782e-01],
        [-5.2022e-02,  3.4757e-02],
        [ 5.1256e-02, -1.5893e-01],
        [ 5.6949e-01,  7.7288e-01],
        [ 3.0758e-01, -1.6004e+00],
        [-1.1578e+00, -1.3398e+00],
        [-1.1509e+00, -1.2968e+00],
        [ 3.4887e-02, -8.9131e-01],
        [-1.0075e-01, -9.5060e-01],
        [ 5.8386e-02, -1.2788e-01],
        [ 1.5273e-01, -8.2237e-01],
        [ 1.8670e-01, -1.8940e-01],
        [ 1.9033e-01,  1.4109e-01],
        [ 1.8284e-01, -2.3942e-01],
        [-1.9974e-02, -1.2614e+00],
        [-1.3311e+00,  1.1877e-01],
        [ 2.1161e+00, -1.0801e-01],
        [-5.0164e-01, -1.3870e+00],
        [ 5.1400e-01, -4.6725e-01],
        [-6.6456e-01,  6.9718e-03],
        [-1.5083e-01,  9.388

In [22]:
print(labels[train_idx].shape)
labels[train_idx]

torch.Size([44, 2])


tensor([[2.4300e-02, 0.0000e+00],
        [0.0000e+00, 1.7200e+00],
        [3.0000e+01, 0.0000e+00],
        [0.0000e+00, 6.4250e+00],
        [2.4075e+01, 1.1900e+03],
        [0.0000e+00, 1.1000e+00],
        [8.0000e-03, 3.0000e-01],
        [1.0000e-03, 7.0000e-03],
        [5.0000e+00, 0.0000e+00],
        [1.0000e+00, 0.0000e+00],
        [1.2000e-02, 8.3000e+01],
        [8.9000e-03, 0.0000e+00],
        [5.0000e-02, 0.0000e+00],
        [0.0000e+00, 5.3000e+00],
        [0.0000e+00, 2.5950e-01],
        [2.1000e+00, 0.0000e+00],
        [1.3500e+01, 6.0000e+01],
        [8.0000e+00, 0.0000e+00],
        [4.2880e-01, 4.9657e+03],
        [8.7000e+00, 0.0000e+00],
        [0.0000e+00, 1.1000e+01],
        [2.0200e+00, 6.2600e+00],
        [5.9000e-02, 0.0000e+00],
        [3.3000e-02, 0.0000e+00],
        [4.0250e-02, 2.5775e+02],
        [5.0000e+00, 4.8500e+01],
        [1.4000e-01, 0.0000e+00],
        [3.0000e+00, 0.0000e+00],
        [3.3000e-04, 4.6000e+00],
        [0.000

In [23]:
# compute the regression loss
def regression_loss(logits, labels):
    return F.mse_loss(logits, labels)

In [24]:
train_loss = regression_loss(logits[train_idx], labels[train_idx])

In [25]:
train_loss

tensor(298309.9688, grad_fn=<MseLossBackward0>)